In [2]:
from lxml import etree
from io import StringIO, BytesIO
import json
import time
import re
import sys
from sys import getsizeof

namespace = '{http://www.mediawiki.org/xml/export-0.10/}'
page_tag = namespace + 'page'
title_tag = namespace + 'title'
redirect_tag = namespace + 'redirect'
text_tag = namespace + 'text'

In [ ]:
# EXTRAER TITULOS NAIVE

start_time = time.time()

context = etree.iterparse('./wiki-dump/enwiki-1gb', events=('end',))

titles = []

for event, elem in context:
    if 'title' in elem.tag:
        if elem.text is not None:
            titles.append(elem.text.lower())
#             titles.append({'title': elem.text.lower(), 'redirect': ''})
#             titles.append((elem.text,))
    elif 'redirect' in elem.tag:
        titles[-1] = elem.attrib['title'].lower()
#         titles[-1]['redirect'] = elem.attrib['title'].lower()
#         titles[-1] = titles[-1] + (elem.attrib['title'],)
    
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context

print('Titles: {} --- {} MB'.format(len(titles), getsizeof(titles) / 1024**2))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()

with open('./wiki-dump/titles', 'w') as outfile:
    json.dump(titles, outfile, indent=2)
    
print("--- %s seconds ---" % (time.time() - start_time))

In [6]:
# EXTRAER TITULOS: ALGORITMO ESTRELLA :D

start_time = time.time()

context = etree.iterparse('./wiki-dump/enwiki-1gb', tag=page_tag , events=('end',))

titles = set()
for event, elem in context:
    try:
        redirect_child = elem.iterchildren(tag=redirect_tag)
        child = redirect_child.__next__()
#         titles.append(child.attrib['title'].lower())
        titles.add(child.attrib['title'].lower())

    except StopIteration:
        title_child = elem.iterchildren(tag=title_tag)
        child = title_child.__next__()
#         titles.append(child.text.lower())
        titles.add(child.text.lower())
        
    elem.clear()
    while elem.getprevious() is not None:
        del elem.getparent()[0]

del context


print('Titles: {} --- {} MB'.format(len(titles), getsizeof(titles) / 1024**2))
print("--- %s seconds ---" % (time.time() - start_time))

Titles: 40554 --- 2.000213623046875 MB
--- 9.96427321434021 seconds ---


In [ ]:
repeated = titles
from collections import Counter
a = dict(Counter(repeated))
print(a)

In [ ]:
count = 1000
for event, elem in etree.iterparse('./wiki-dump/enwiki-1gb', events=('start', 'end')):
    if event == 'start' and 'page' in elem.tag:
        f = elem.find('{http://www.mediawiki.org/xml/export-0.10/}redirect')
        if f is not None:
            print(f.attrib['title'])
        else:
            f = elem.find('{http://www.mediawiki.org/xml/export-0.10/}title')
            print(f.text)
    if count <= 0:
        break
    count -= 1
    elem.clear()
        

In [ ]:
count = 170
text = ''
for event, elem in etree.iterparse('./wiki-dump/enwiki-1gb', events=('start', 'end')):
    if event == 'start' and 'text' in elem.tag:
        count -= 1
        if count == 0:
            text = elem.text
            break

# m = re.findall('\[\[(.*?)]]', text)
m = re.findall('\[\[([^][]+)]]', text)
print(len(m))
print(m)

for i, s in enumerate(m):
    m[i] = s.split('|')[-1]

print(m)


In [ ]:
print(text)


In [ ]:
vocabulary = []
alternative_keyphrases = set()

start_time = time.time()

# with open('./wiki-dump/titles', 'wa') as f:

# Extract keyphrases from articles
for event, elem in etree.iterparse('./wiki-dump/enwiki-20160113-pages-articles.xml', events=('start', 'end')):
    title = ''
    # Extract titles of pages
    if event == 'start' and 'title' in elem.tag:
        if elem.text is not None:
#                 vocabulary.append(elem.text.lower())
#                 title = elem.text.lower()
            pass

    elif event == 'start' and 'redirect' in elem.tag:
#             vocabulary[-1] = elem.attrib['title'].lower()
#             title = elem.attrib['title'].lower()
        pass

    # Extract keyphrases in other morphological form
    elif event == 'start' and 'text' in elem.tag:
        text = elem.text
        if text is not None:
#                 keyphrases = re.findall('\[\[([^][]+)]]', text)
#                 for key in keyphrases:
#                     alternative_keyphrases.add(key.split('|')[-1].lower())
            pass

    elem.clear()
        
    
    
print("--- %s seconds ---" % (time.time() - start_time))


start_time = time.time()
vocabulary = alternative_keyphrases.union(set(vocabulary))
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
print('Vocabulary: ', len(vocabulary))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Open corpus
context = etree.iterparse('./wiki-dump/enwiki-20160113-pages-articles.xml', events=('start', 'end'))

# Extract keyphrases from articles
def extract_keyphrases(even, elem):
    title = ''
    # Extract titles of pages
    if event == 'start' and 'title' in elem.tag:
        if elem.text is not None:
#                 vocabulary.append(elem.text.lower())
#                 title = elem.text.lower()
            pass

    elif event == 'start' and 'redirect' in elem.tag:
#             vocabulary[-1] = elem.attrib['title'].lower()
#             title = elem.attrib['title'].lower()
        pass

    # Extract keyphrases in other morphological form
    elif event == 'start' and 'text' in elem.tag:
        text = elem.text
        if text is not None:
#                 keyphrases = re.findall('\[\[([^][]+)]]', text)
#                 for key in keyphrases:
#                     alternative_keyphrases.add(key.split('|')[-1].lower())
            pass
    

def fast_iter(context, func):
    for event, elem in context:
        func(event, elem)
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    del context

In [ ]:
s = set()
for i in range(5000000):
    s.add('Un string asadas ' + str(i))
    
getsizeof(s) / (1024*1024)